In [1]:
from __future__ import print_function
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib.tensorboard.plugins import projector

In [2]:
# number 1 to 10 data 输入数据
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

#图片数量
image_num=3000

sess=tf.Session()

#文件路径 （程序所在路径）
DIR="/Users/yunhuarong/"

#初始化变量
init=tf.global_variables_initializer()

#载入图片
embedding=tf.Variable(tf.stack(mnist.test.images[:image_num]),trainable=False,name='embedding')

#参数概要
def variable_summaries(var):
    with tf.name_scope('summaires'):
        mean=tf.reduce_mean(var)
        #平均值
        tf.summary.scalar('mean',mean)
        with tf.name_scope('stddev'):
            stddev=tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
        tf.summary.scalar('stddev',stddev)
        tf.summary.scalar('max',tf.reduce_max(var))
        tf.summary.scalar('min',tf.reduce_min(var))
        tf.summary.histogram('histogram',var)


#定义层
with tf.name_scope('layer'):

    def add_layer(inputs, in_size, out_size, activation_function=None):
    # add one more layer and return the output of this layer
        with tf.name_scope('Weights'):
            Weights = tf.Variable(tf.random_normal([in_size, out_size]))
        with tf.name_scope('biases'):
            biases = tf.Variable(tf.zeros([1, out_size]) + 0.1,)
        with tf.name_scope('w_x_plus_b'):
            Wx_plus_b = tf.matmul(inputs, Weights) + biases
        if activation_function is None:
            outputs = Wx_plus_b
        else:
            outputs = activation_function(Wx_plus_b,)
        return outputs


with tf.name_scope('accuracy'):    
    def accuracy(v_xs, v_ys):
        global prediction
        y_pre = sess.run(prediction, feed_dict={xs: v_xs})
        with tf.name_scope('correct_prediction'):
            correct_prediction = tf.equal(tf.argmax(y_pre,1), tf.argmax(v_ys,1))
        with tf.name_scope('accuracy'):
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
            tf.summary.scalar('accuracy',accuracy)
            result = sess.run(accuracy, feed_dict={xs: v_xs, ys: v_ys})
        return result

    
with tf.name_scope('input'):
# define placeholder for inputs to network
    xs = tf.placeholder(tf.float32, [None, 784],name='input_xs') # 28x28
    ys = tf.placeholder(tf.float32, [None, 10],name='input_ys')

#显示图片
with tf.name_scope('input_reshape'):
        image_shaped_input=tf.reshape(xs,[-1,28,28,1])
        tf.summary.image('input_image',image_shaped_input,10)    

# add output layer
with tf.name_scope('prediction')
    prediction = add_layer(xs, 784, 10, activation_function=tf.nn.softmax)

with tf.name_scope('cross_entropy'):
# the error between prediction and real data
    cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys * tf.log(prediction),reduction_indices=[1]))   
    tf.summary.scalar('cross_entropy',cross_entropy)
    
with tf.name_scope('train'):
# loss
    train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

#产生metadata文件
#检查指定路径是否有此文件，如果有删除
if tf.gfile.Exists(DIR+'projector/projector/metadata.tsv'):
    tf.gfile.DeleteRecursively(DIR+'projector/projector/metadata.tsv')
with open(DIR+'/projector/projector/metadata.tsv','w')as f:
    #查找1所在位置，提取位置就是标签
    labels=sess.run(tf.argmax(mnist.test.labels[:],1))    
#循环
    for i in range(image_num):
            #每次写入后隔开一行
            f.write(str(labels[i])+'\n')
            
#合并所有summaries
merged=tf.summary.merge_all()

#官网visualization例子
projector_writer=tf.summary.FileWriter(DIR+'projector/projector',sess.graph)
saver=tf.train.Saver()
config=projector.ProjectorConfig()
embed=config.embeddings.add()
embed.tensor_name=embedding.name
embed.metadata_path=DIR +'projector/projector/metadata.tsv'
embed.sprite.image_path=DIR +'projector/data/mnist_10k_sprite.png'
embed.sprite.single_image_dim.extend([28,28])
projector.visualize_embeddings(projector_writer,config)
    
sess = tf.Session()
# important step
with tf.Session() as sess:
    sess.run(init)
    writer=tf.summary.FileWriter('logs/tensorboard_test/test/1',sess.graph)
    for i in range(1000):
        batch_xs,batch_ys=mnist.train.next_batch(100)
        run_options=tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
        run_metadata=tf.RunMetadata() 
        summary,_=sess.run([merged,train_step],feed_dict={xs:batch_xs,ys:batch_ys},options=run_options,run_metadata=run_metadata)
        projector_writer.add_run_metadata(run_metadata,'step%03d'% i)
        projector_writer.add_summary(summary,i)
    
        if i%100 == 0:
                acc=sess.run(accuracy,feed_dict={xs:mnist.test.images,ys:mnist.test.labels})
                print("Iter"+str(i)+", Testing accuracy="+str(acc))
            
    saver.save(sess,DIR +'projector/projector/a_model.ckpt',global_step=image_num)
    projector_writer.close()
sess.close()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


FailedPreconditionError: Attempting to use uninitialized value Weights/Variable
	 [[Node: Weights/Variable/read = Identity[T=DT_FLOAT, _class=["loc:@Weights/Variable"], _device="/job:localhost/replica:0/task:0/cpu:0"](Weights/Variable)]]

Caused by op 'Weights/Variable/read', defined at:
  File "/anaconda3/envs/tensoflow_session/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/anaconda3/envs/tensoflow_session/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/anaconda3/envs/tensoflow_session/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/anaconda3/envs/tensoflow_session/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/anaconda3/envs/tensoflow_session/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 486, in start
    self.io_loop.start()
  File "/anaconda3/envs/tensoflow_s